In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def score_dataset(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return mean_absolute_error(y_valid, preds)

In [85]:
import pandas as pd

df = pd.read_csv('melb_data.csv')

missing_values_count = df.isnull().sum()


In [86]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.7, random_state=42)

1. Jakie oceny zostały uzyskane dla następujących strategii usuwania/uzupełniania danych:

a) usunięcie wierszy


In [87]:
# czyszczenie danych
train_df_cleaned = train_df.dropna()
test_df_cleaned = test_df.dropna()

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

188354.82856287423


b) usunięcie kolumn

In [88]:
# czyszczenie danych
train_df_cleaned = train_df.dropna(axis=1)
test_df_cleaned = test_df.dropna(axis=1)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

196311.7920648613


c) uzupełnienie braków 0

In [89]:
# czyszczenie danych
train_df_cleaned = train_df.fillna(0)
test_df_cleaned = test_df.fillna(0)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

188226.58769531525


d) uzupełnienie braków wartością sąsiednią

In [90]:
# czyszczenie danych
train_df_cleaned = train_df.fillna(method='bfill', axis=0).fillna(0)
test_df_cleaned = test_df.fillna(method='bfill', axis=0).fillna(0)

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

192773.70847569956


e) uzupełnienie braków medianą

In [91]:
# czyszczenie danych
train_df_cleaned = train_df.fillna(train_df.median(numeric_only=True))
test_df_cleaned = test_df.fillna(test_df.median(numeric_only=True))

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

188225.78676622786


f) dodatnie dnia tygodnia kiedy nastąpiła sprzedaż

In [92]:
train_df_cleaned = train_df.fillna(train_df.median(numeric_only=True)).copy()
test_df_cleaned = test_df.fillna(test_df.median(numeric_only=True)).copy()

train_df_cleaned['Day of week'] = pd.to_datetime(train_df.loc[:, "Date"], dayfirst=True).dt.dayofweek
test_df_cleaned['Day of week'] = pd.to_datetime(test_df.loc[:, "Date"], dayfirst=True).dt.dayofweek

cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

188589.44182864457


g) dodanie nowej cechy (np. CouncilArea) za pomocą LabelEncoder

In [93]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
train_df_cleaned = train_df.fillna(train_df.median(numeric_only=True)).copy()
test_df_cleaned = test_df.fillna(test_df.median(numeric_only=True)).copy()

label_encoder = LabelEncoder()
col='CouncilArea'

label_encoder.fit(df[col])

train_df_cleaned[col] = label_encoder.transform(train_df_cleaned[col])
test_df_cleaned[col] = label_encoder.transform(test_df_cleaned[col])


cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

188082.05119478423


h) dodanie nowej cechy (np. CouncilArea) za pomocą BinaryEncoder

In [94]:
from sklearn.preprocessing import LabelBinarizer
    

train_df_cleaned = train_df.fillna(train_df.median(numeric_only=True)).copy()
test_df_cleaned = test_df.fillna(test_df.median(numeric_only=True)).copy()

col='CouncilArea'
df_copy = df.copy().fillna(df[col].mode()[0]).replace('Unavailable', 'Unknown')

train_df_cleaned = train_df.fillna(train_df['CouncilArea'].mode()[0]).replace('Unavailable', 'Unknown').copy()
test_df_cleaned = test_df.fillna(test_df['CouncilArea'].mode()[0]).replace('Unavailable', 'Unknown').copy()


label_binarizer = LabelBinarizer()


label_binarizer.fit(df_copy[col])

train_df_cleaned[col] = label_binarizer.transform(train_df_cleaned[col])
test_df_cleaned[col] = label_binarizer.transform(test_df_cleaned[col])


cols_x = train_df_cleaned.select_dtypes(include=[np.number]).columns.difference(['Price'])  # wybiera tylko kolumny z wartosciami numerycznymi, za wyjątkiem kolumny z wartością referencyjną - wejście do klasyfikatora
cols_y = 'Price'  # - wyjście z klasyfikatora
print(score_dataset(train_df_cleaned[cols_x], test_df_cleaned[cols_x], train_df_cleaned[cols_y], test_df_cleaned[cols_y]))

195773.68117271797


2. Spróbuj odpowiedzieć na pytanie: które podejście/podejścia wydają się być najbardziej obiecujące w odniesieniu do estymacji cen nieruchomości (w odniesieniu do uzupełniania braków w poszczególnych kolumnach oraz tworzenia nowych cech bazujących na zmiennych nominalnych?


Podeściami które najbardziej rokują na poprawienie estymacji ceny nieruchomości jest połączenie uzupełnienia pustych wartości medianą lub wartością średnią oraz dodanie nowych cech, na przykład odpowiadających za rejon (Council Area). Mimo niewielkiej poprawy wyników, wart byłoby wprowadzić analizę istotności cech, tak aby zredukować wymiarowość zbioru, być może wtedy okaże się że zakodowane dane o rejonie są istotne, a uda nam się zredukować kolumny mniej istotne, co pozytywnie wpłynie na wynik.